## Computing with NumPy arrays

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('../chapter2/data/nyc_data.csv',
                   parse_dates=['pickup_datetime', 'dropoff_datetime'])

In [3]:
pickup = data[['pickup_longitude', 'pickup_latitude']].values
dropoff = data[['dropoff_longitude', 'dropoff_latitude']].values
pickup

array([[-73.955925,  40.781887],
       [-74.005501,  40.745735],
       [-73.969955,  40.79977 ],
       ...,
       [-73.993492,  40.729347],
       [-73.978477,  40.772945],
       [-73.987206,  40.750568]])

### Selection and indexing

In [4]:
print(pickup[3, 1])

40.755081

In [5]:
pickup[1:7:2, 1:]

array([[ 40.745735],
       [ 40.755081],
       [ 40.768978]])

In [6]:
lon = pickup[:, 0]
lon

array([-73.9559, -74.0055, ..., -73.9784, -73.9872])

In [7]:
lat = pickup[:, 1]
lat

array([ 40.7818, 40.7457, ..., 40.7729, 40.7505])

### Boolean operations on arrays

In [8]:
lon_min, lon_max = (-73.98330, -73.98025)
lat_min, lat_max = ( 40.76724,  40.76871)

In [9]:
in_lon = (lon_min <= lon) & (lon <= lon_max)
in_lon

array([False, False, False, ..., False, False, False], dtype=bool)

In [10]:
in_lon.sum()

69163

In [11]:
in_lat = (lat_min <= lat) & (lat <= lat_max)

In [12]:
in_lonlat = in_lon & in_lat
in_lonlat.sum()

3998

In [13]:
np.nonzero(in_lonlat)[0]

array([   901,   1011,   1066, ..., 845749, 845903, 846080])

In [14]:
lon1, lat1 = dropoff.T

### Mathematical operations on arrays

In [15]:
EARTH_R = 6372.8
def geo_distance(lon0, lat0, lon1, lat1):
    """Return the distance (in km) between two points in
    geographical coordinates."""
    # from: http://en.wikipedia.org/wiki/Great-circle_distance
    # and: http://stackoverflow.com/a/8859667/1595060
    lat0 = np.radians(lat0)
    lon0 = np.radians(lon0)
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    dlon = lon0 - lon1
    y = np.sqrt(
        (np.cos(lat1) * np.sin(dlon)) ** 2
         + (np.cos(lat0) * np.sin(lat1)
         - np.sin(lat0) * np.cos(lat1) * np.cos(dlon)) ** 2)
    x = np.sin(lat0) * np.sin(lat1) + \
        np.cos(lat0) * np.cos(lat1) * np.cos(dlon)
    c = np.arctan2(y, x)
    return EARTH_R * c

In [16]:
distances = geo_distance(lon, lat, lon1, lat1)

In [17]:
plt.hist(distances[in_lonlat], np.linspace(0., 10., 50))
plt.xlabel('Trip distance (km)')
plt.ylabel('Number of trips')

### A density map with NumPy

In [18]:
evening = (data.pickup_datetime.dt.hour >= 19).values

In [19]:
n = np.sum(evening)

In [20]:
n

242818

In [21]:
weights = np.zeros(2 * n)

In [22]:
weights[:n] = -1
weights[n:] = +1

In [23]:
points = np.r_[pickup[evening],
               dropoff[evening]]

In [24]:
points.shape

(485636, 2)

In [25]:
def lat_lon_to_pixels(lat, lon):
    lat_rad = lat * np.pi / 180.0
    lat_rad = np.log(np.tan((lat_rad + np.pi / 2.0) / 2.0))
    x = 100 * (lon + 180.0) / 360.0
    y = 100 * (lat_rad - np.pi) / (2.0 * np.pi)
    return (x, y)

In [26]:
lon, lat = points.T
x, y = lat_lon_to_pixels(lat, lon)

In [27]:
lon_min, lat_min = -74.0214, 40.6978
lon_max, lat_max = -73.9524, 40.7982

In [28]:
x_min, y_min = lat_lon_to_pixels(lat_min, lon_min)
x_max, y_max = lat_lon_to_pixels(lat_max, lon_max)

In [29]:
bin = .00003
bins_x = np.arange(x_min, x_max, bin)
bins_y = np.arange(y_min, y_max, bin)

In [30]:
grid, _, _ = np.histogram2d(y, x, weights=weights,
                            bins=(bins_y, bins_x))

In [31]:
density = 1. / (1. + np.exp(-.5 * grid))

In [32]:
plt.imshow(density,
           origin='lower',
           interpolation='bicubic'
           )
plt.axis('off')

### Other topics